## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [9]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

#cv2.destroyAllWindows()

sample_img = cv2.imread(images[0])

def calibrate(img, objpoints, imgpoints):
    #Grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    #calibrate
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return mtx, dist

def undistort(img, mtx, dist):
    #undistort
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    return undist

mtx, dist = calibrate(sample_img, objpoints, imgpoints)
dst = undistort(sample_img, mtx, dist)

plt.figure(figsize=(7,12))
plt.imshow(sample_img)

plt.figure(figsize=(7,12))
plt.imshow(dst)

## Undistort Raw Images...

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

undist_output = 'p4.mp4'
clip1 = VideoFileClip("project_video.mp4")
undist_clip = clip1.fl_image(undistort) #NOTE: this function expects color images!!
%time undist_clip.write_videofile(undist_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(undist_output))

## Apply Color Transforms and Gradient Thresholds

In [ ]:
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    s_channel = hls[:,:,2]
    # Sobel x
    #sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)#, ksize=3)
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    return color_binary

# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements

# Make a list of calibration images
images = glob.glob('test_images/test*.jpg')

for fname in images:
    img = cv2.imread(fname)
    image = undistort(img, mtx, dist) #undistort each new frame
    
    cv2.imshow('img',img)
    cv2.waitKey(2000)
    
    cv2.imshow('img',pipeline(image))
    cv2.waitKey(2000)

    cv2.destroyAllWindows()

## Perspective Transform

In [53]:
# Make a straight line images
images = glob.glob('test_images/straight_lines*.jpg')

for fname in images:
    img = cv2.imread(fname)
    image = undistort(img, mtx, dist) #undistort each new frame

    cv2.imshow('img',image)
    cv2.waitKey(2000)
    
    #plt.figure(figsize=(7,12))
    #plt.imshow(image)
    
    src = np.float32([(546, 480), (738, 480), (200,700), (1100, 700)])
    # define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    dest = np.float32([(50, 0), (550, 0), (50, 700), (550, 700)])
    # use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dest)
    # use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(image, M, image.shape[0:2], flags=cv2.INTER_LINEAR)
    
    cv2.imshow('img',warped)
    cv2.waitKey(2000)